# Week 8 — Introduction to Web Scraping

## Learning ObjectivesBy the end of this session, you will be able to:
* Fetch web pages using Python's urllib standard library
* Parse HTML content using html.parser from the standard library
* Extract links and simple data from HTML documents
* Handle common web scraping errors and edge cases
* Understand the ethical considerations of web scraping
* Apply web scraping techniques to gather data for analysis

### 🔄 JavaScript Transition Note

If you're coming from JavaScript/web development, you're probably familiar with the Fetch API or axios for making HTTP requests, and libraries like Cheerio or built-in DOM methods for HTML parsing. Python's urllib provides similar functionality to fetch(), but with a more traditional callback-style API. The html.parser works similarly to DOM parsing, but with a different event-driven approach using handle_starttag() and other methods instead of query selectors. Both approaches achieve the same goal: extracting data from HTML documents.

### Fetch with urllib

In [ ]:
from urllib.request import urlopenfrom urllib.error import URLErrordef fetch(url):    try:        with urlopen(url) as resp:            return resp.read().decode('utf-8')    except URLError as e:        return ''html = '<html><body><a href="https://example.com">example</a></body></html>'print(len(html))

**Activity:** Fetch a known URL or use a provided HTML string and print its length.

In [ ]:
pass

### Parse Links (html.parser)

In [ ]:
from html.parser import HTMLParserclass LinkParser(HTMLParser):    def __init__(self):        super().__init__()        self.links = []    def handle_starttag(self, tag, attrs):        if tag == 'a':            for k,v in attrs:                if k == 'href':                    self.links.append(v)p = LinkParser()p.feed('<a href="https://a">A</a> <a href="/b">B</a>')print(p.links)

**Activity:** Parse links from HTML content and filter to absolute URLs only.

In [ ]:
pass

### RecapStdlib can fetch and parse simple HTML when external libraries are unavailable.